In [116]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
import ast
import csv

# step 0: load the data

In [117]:
df = pd.DataFrame()
df2 = pd.read_csv("track_features_lst2.csv")
df2.describe()

,"{'danceability': 0.55, 'energy': 0.5, 'key': 7, 'loudness': -5.779, 'mode': 0, 'speechiness': 0.045, 'acousticness': 0.285, 'instrumentalness': 0, 'liveness': 0.116, 'valence': 0.351, 'tempo': 115.016, 'type': 'audio_features', 'id': '7oQepKHmXDaPC3rgeLRvQu', 'uri': 'spotify:track:7oQepKHmXDaPC3rgeLRvQu', 'track_href': 'https://api.spotify.com/v1/tracks/7oQepKHmXDaPC3rgeLRvQu', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7oQepKHmXDaPC3rgeLRvQu', 'duration_ms': 167357, 'time_signature': 4}"
count,10003
unique,8050
top,"{'danceability': 0.707, 'energy': 0.681, 'key'..."
freq,12


In [146]:
with open('track_features_lst2.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    data = list(reader)

In [185]:
df = pd.DataFrame.from_dict(data[0])

ValueError: If using all scalar values, you must pass an index

In [183]:
df

,0
0,"{'danceability': 0.792, 'energy': 0.679, 'key'..."


In [173]:
df = pd.DataFrame()
df.append(data[0].keys(), ignore_index=0)

C:\Users\Phylanx\AppData\Local\Temp\ipykernel_13056\2988416344.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(data[0].keys(), ignore_index=0)


TypeError: cannot concatenate object of type '<class 'dict_keys'>'; only Series and DataFrame objs are valid

In [22]:
# make each row with str elements into dicts and assign them to the corresponding features
df_final = pd.DataFrame(columns=["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"])
for i in range(len(df_final)):
    try:
        df_final.loc[i] = ast.literal_eval(df2.loc[i][0])
    except:
        pass

In [108]:
# make each row with str elements into dicts and assign them to the corresponding features
df_final = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
for i in range(len(df2)):
    try:
        df_final = pd.concat([[df_final, pd.DataFrame.from_dict(ast.literal_eval(df2.loc[i][0]))]], axis=0)
    except:
        pass

In [115]:
df_final = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

pd.concat([df_final, pd.DataFrame.from_dict(ast.literal_eval(df2.loc[i][0])], axis=0)

ValueError: If using all scalar values, you must pass an index

In [107]:
ast.literal_eval(df2.loc[0][0])

{'danceability': 0.792,
 'energy': 0.679,
 'key': 9,
 'loudness': -5.516,
 'mode': 0,
 'speechiness': 0.041,
 'acousticness': 0.517,
 'instrumentalness': 1.49e-05,
 'liveness': 0.0962,
 'valence': 0.204,
 'tempo': 119.932,
 'type': 'audio_features',
 'id': '7DyDjhZMEIK5Ied4juTCyc',
 'uri': 'spotify:track:7DyDjhZMEIK5Ied4juTCyc',
 'track_href': 'https://api.spotify.com/v1/tracks/7DyDjhZMEIK5Ied4juTCyc',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7DyDjhZMEIK5Ied4juTCyc',
 'duration_ms': 151320,
 'time_signature': 4}

In [84]:
pd.DataFrame.from_dict(ast.literal_eval(df2.loc[0][0]))

ValueError: If using all scalar values, you must pass an index

In [70]:
test = pd.DataFrame([["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]])

In [71]:
test

,0,1,2,3,4,5,6,7,8,9,10
0,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms


In [73]:
df_final = pd.DataFrame(columns=["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"])
pd.concat([df_final, pd.DataFrame(test)], axis=0)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,...,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms


In [26]:
ast.literal_eval(df2.loc[0][0])

{'danceability': 0.792,
 'energy': 0.679,
 'key': 9,
 'loudness': -5.516,
 'mode': 0,
 'speechiness': 0.041,
 'acousticness': 0.517,
 'instrumentalness': 1.49e-05,
 'liveness': 0.0962,
 'valence': 0.204,
 'tempo': 119.932,
 'type': 'audio_features',
 'id': '7DyDjhZMEIK5Ied4juTCyc',
 'uri': 'spotify:track:7DyDjhZMEIK5Ied4juTCyc',
 'track_href': 'https://api.spotify.com/v1/tracks/7DyDjhZMEIK5Ied4juTCyc',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7DyDjhZMEIK5Ied4juTCyc',
 'duration_ms': 151320,
 'time_signature': 4}

In [23]:
X = pd.DataFrame(df_final)
X

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms


# step 1: clean the data

In [ ]:
X.index=X["id"]

In [ ]:
X = X.drop("id",axis=1)

In [ ]:
X.describe()

# step 2: scale the data

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns, index=X.index)
display(X.head())
print()
display(X_scaled_df.head())

# save scaler in pickle file
with open("Model/scaler_38k.pkl", "wb") as f:
    pickle.dump(scaler,f)

In [ ]:
X_scaled_df.describe()

# step 3: cluster the data

## assign k=8 as default

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=1234)
kmeans.fit(X_scaled_df)

In [ ]:
kmeans.labels_

In [ ]:
# assign a cluster to each example
labels = kmeans.predict(X_scaled_df)
# retrieve unique clusters
clusters = np.unique(labels)
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = np.where(labels == cluster)
    # create scatter of these samples
    pyplot.scatter(X.to_numpy()[row_ix, 1], X.to_numpy()[row_ix, 3])
    # show the plot
pyplot.show()

In [ ]:
kmeans8 = KMeans(n_clusters=8,
                init="k-means++",
                n_init=50,  # try with 1, 4, 8, 20, 30, 100...
                max_iter=1,
                tol=0,
                algorithm="elkan",
                random_state=1234)
kmeans8.fit(X_scaled_df)
print(kmeans8.inertia_)

## now find the optimal k

## try elbow

In [ ]:
K = range(2, 21)
inertia = []

for k in K:
    print("Training a K-Means model with {} clusters! ".format(k))
    print()
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_scaled_df)
    inertia.append(kmeans.inertia_)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

## try silhouette

In [ ]:
K = range(2, 20)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_scaled_df)

    filename = "Model/kmeans_" + str(k) + ".pickle"
    with open(filename, "wb") as f:
        pickle.dump(kmeans,f)

    silhouette.append(silhouette_score(X_scaled_df, kmeans.predict(X_scaled_df)))


plt.figure(figsize=(16,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Method showing the optimal k')

In [ ]:
len(X_scaled_df.columns)

# step 4: apply optimal model

In [ ]:
kmeans15 = KMeans(n_clusters=15,
                init="k-means++",
                n_init=50,  # try with 1, 4, 8, 20, 30, 100...
                max_iter=1,
                tol=0,
                algorithm="elkan",
                random_state=1234)
kmeans15.fit(X_scaled_df)
print(kmeans15.inertia_)

In [ ]:
with open("Model/kmeans_15.pkl", "wb") as f:
    pickle.dump(kmeans15,f)

In [ ]:
X_scaled_df["cluster"] = kmeans15.labels_

## assign labels

In [ ]:
X_scaled_df

In [ ]:
df_clustered_15 = X_scaled_df["cluster"]

# step 5: save to .csv

In [ ]:
df_clustered_15.to_csv("df_clustered_15.csv", index_label=False)